In [1]:
import sys

print("Python version:", sys.version)

Python version: 3.13.9 | packaged by conda-forge | (main, Oct 16 2025, 10:23:36) [MSC v.1944 64 bit (AMD64)]


In [2]:
import os
from agentpro import ReactAgent
from agentpro.tools import AresInternetTool
from agentpro import create_model

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#Create custom tool for agentPro
from agentpro.tools import Tool
from typing import Any

class MyCustomTool(Tool):
    name: str = "Local RAG Tool"  # Human-readable name for the tool (used in documentation and debugging)
    description: str = "Contains most up to date information on everything cats, its use should be prioritized if question is related to cats."  # Brief summary explaining the tool's functionality for agent
    action_type: str = "local_rag"  # Unique identifier for the tool; lowercase with underscores for agent; avoid spaces, digits, special characters
    input_format: str = "A string query"  # Instruction on what kind of input the tool expects with example

    def run(self, input_text: Any) -> str:
        # Load FAISS vectorstore and perform RAG retrieval
        from langchain_community.vectorstores import FAISS
        from langchain_huggingface import HuggingFaceEmbeddings
        INDEX_PATH = "RAG/faiss_index"
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vectorstore = FAISS.load_local(INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
        results = vectorstore.similarity_search(input_text, k=4)
        return "\n".join([r.page_content for r in results])


In [11]:
# Create a model with OpenAI
model = create_model(provider="openai", model_name="gpt-4.1-nano", api_key=os.getenv("OPENAI_API_KEY", None))

# Initialize tools
tools = [
    AresInternetTool(os.getenv("ARES_API_KEY", None)),
    MyCustomTool()
]

# Initialize agent
agent = ReactAgent(model=model, tools=tools)

# Run a query
query = "Tell me about chlorine in fresh tap water and cats"
response = agent.run(query)

print(f"\nFinal Answer: {response.final_answer}")

================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: ares_internet_search, local_rag

Tool: Ares Internet Search
Description: Uses Ares API to search live and detailed information from the internet and returns a clean summary and related links.
Action Type: ares_internet_search
Input Format: A search query as a string. Example: 'Best restaurants in San Francisco'


Tool: Local RAG Tool
Description: Contains most up to date information on everything cats, its use should be prioritized.
Action Type: local_rag
Input Format: A string query


Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the fin